In [12]:
import numpy as np
import pandas as pd


<h2>Creating Functions which can be used on datasets. </h2>

-Assuming the dataset has price value for each day and each month is 30 days <br>
-Taking Indian Rupee 3 month MIBOR rate as 7.3 default

In [13]:
def calculate_momentum(data, delta1=7, delta2=13, lsfr=7.3):
    pt_1 = data.iloc[-1*30]  
    pt_7 = data.iloc[-delta1*30]  
    pt_13 = data.iloc[-delta2*30]

    pm6 = ((pt_1/pt_7)-1) - lsfr

    pm12 = np.nan
    if not np.isnan(pt_13):
        pm12 = ((pt_1/pt_13)-1) - lsfr

    else:
        pm12 = pm6
    
    print(f'6 month Price Momentum : {pm6} ')
    print(f'12 month Price Momentum : {pm12}')

    return pm6, pm12

In [14]:
def risk_adj_pm(pm,d):
    std1 = np.std(d.iloc[-3*365])

    rapm = pm/std1

    return rapm
    

Calculating Momentum Score


In [15]:
def momentum_score(d,pm6_mean,pm6_std,pm12_mean,pm12_std,cmean,cstd):
    pm6 , pm12 = calculate_momentum(d)
    
    pm6z = (pm6-pm6_mean)/pm6_std
    pm12z = (pm12-pm12_mean)/pm12_std

    C = (pm6z*0.5)+(pm12z*0.5)

    Z = (C-cmean)/cstd

    if z>3:
        z=3
    elif z<-3:
        z=-3

    if z>0:
        momentumscore = 1+Z
    elif z<0:
        momentumscore = 1/(1-z)
        
    return z,momentumscore

Security Selection

In [16]:
def security_selection(d,momentum_score,parent_index,n):

    combd = pd.DataFrame(d,{"Momentum Z Score":momentum_score,"Parent Index Weight": parent_index}) 
    sorted = combd.sort_values(by=['Momentum Z Score','Parent Index Weight'], ascending = [False, False])

    selected_securities = sorted.head(n).index.tolist()

    return selected_securities

In [17]:
def Momentum_weight(d, market_cap,security_market_caps,cap_level):
    score = momentum_score(d)
    mo_wt = score*market_cap
    normalized_weights = mo_wt / mo_wt.sum()
    inclusion_factors = normalized_weights / (security_market_caps / security_market_caps.sum())
    inclusion_factors = inclusion_factors.clip(upper=cap_level)
    return inclusion_factors